In [1]:
import tweepy
import pandas as pd

In [2]:
consumer_key = '****'
consumer_secret = '****'
access_token_key = '****'
access_token_secret = '****'

# Authenticate to Twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token_key, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


In [3]:
# Workaround the API limit of 100 tweets per request

def get_tweets(search_keywords, max_tweets):
    
    df = pd.DataFrame()
    max_tweets = max_tweets

    for keyword in search_keywords:
        
        since_id = None
        max_id = 0
        tweet_count = 0
        request_number = 1
        
        while tweet_count < max_tweets:
            try:
                
                if (max_id == 0): 
                    
                    if (not since_id):
                        tweets = api.search(q=keyword+'-filter:retweets -filter:replies', 
                                            lang='en', 
                                            count=100,
                                            tweet_mode='extended',
                                            show_user=True)
                    else:
                        tweets = api.search(q=keyword+'-filter:retweets -filter:replies', 
                                            lang='en', 
                                            count=100, 
                                            show_user=True,
                                            tweet_mode='extended',
                                            since_id=since_id)
                    
                else:
                    
                    if (not since_id):
                        tweets = api.search(q=keyword+'-filter:retweets -filter:replies', 
                                            lang='en', 
                                            count=100, 
                                            show_user=True,
                                            tweet_mode='extended',
                                            max_id=str(max_id - 1))
                    else:
                        tweets = api.search(q=keyword+'-filter:retweets -filter:replies', 
                                            lang='en', 
                                            count=100, 
                                            show_user=True,
                                            tweet_mode='extended',
                                            max_id=str(max_id - 1),
                                            since_id=since_id)
                    
                # save tweets into dictionary and append to dataframe
                tweets_dict = [{'text':status.full_text, 
                                'id':status.id, 
                                'query':keyword, 
                                'created_at':status.created_at,
                                'hashtags':status.entities.get('hashtags'),
                                'symbols':status.entities.get('symbols'),
                                'retweet_count':status.retweet_count,
                                'favorite_count':status.favorite_count,
                                'in_reply_to_user_id':status.in_reply_to_user_id,
                                'label':None} for status in tweets if not status.in_reply_to_user_id]
                
                df = df.append(tweets_dict, ignore_index=True)
                
                max_id = tweets[-1].id
                tweet_count += len(tweets)
                
                print('Retrieved ' + str(len(tweets)) + ' tweets during Request # {}'.format(request_number) + ' for query ' + keyword)
                request_number += 1
                
            except:
                print('No more tweets!')
                break
            
    return df

In [4]:
search_terms = ['tesla']
max_tweets_per_query = 10000
tweets_df = get_tweets(search_terms, max_tweets_per_query)

Retrieved 100 tweets during Request # 1 for query tesla
Retrieved 100 tweets during Request # 2 for query tesla
Retrieved 100 tweets during Request # 3 for query tesla
Retrieved 100 tweets during Request # 4 for query tesla
Retrieved 100 tweets during Request # 5 for query tesla
Retrieved 100 tweets during Request # 6 for query tesla
Retrieved 100 tweets during Request # 7 for query tesla
Retrieved 100 tweets during Request # 8 for query tesla
Retrieved 100 tweets during Request # 9 for query tesla
Retrieved 100 tweets during Request # 10 for query tesla
Retrieved 100 tweets during Request # 11 for query tesla
Retrieved 100 tweets during Request # 12 for query tesla
Retrieved 100 tweets during Request # 13 for query tesla
Retrieved 100 tweets during Request # 14 for query tesla
Retrieved 100 tweets during Request # 15 for query tesla
Retrieved 100 tweets during Request # 16 for query tesla
Retrieved 100 tweets during Request # 17 for query tesla
Retrieved 100 tweets during Request # 18

In [5]:
tweets_df.head()

,text,id,query,created_at,hashtags,symbols,retweet_count,favorite_count,in_reply_to_user_id,label
0,New video of Tesla Model S shows just how awfu...,1405281806509772800,tesla,2021-06-16 21:51:06,"[{'text': 'malysia', 'indices': [125, 133]}, {...",[],0,0,None,None
1,Welcome back to the Tesla family 👍 https://t.c...,1405281717816938500,tesla,2021-06-16 21:50:45,[],[],1,1,None,None
2,This week’s #RucaNews 👇🏻\n“Tesla works with gl...,1405281684946227202,tesla,2021-06-16 21:50:37,"[{'text': 'RucaNews', 'indices': [12, 21]}]",[],0,0,None,None
3,Bitch put gold D’s on a Tesla the most top tie...,1405281681930522630,tesla,2021-06-16 21:50:36,[],[],0,0,None,None
4,Number of Teslas registered in Norway today wa...,1405281540121141253,tesla,2021-06-16 21:50:02,"[{'text': 'Tesla', 'indices': [153, 159]}, {'t...","[{'text': 'TSLA', 'indices': [168, 173]}]",0,1,None,None


In [6]:
tweets_df.shape

(10016, 10)

In [7]:
tweets_df.to_csv('raw_tweets_tesla.csv', index=False)